# 跳表

## 原理
> 已知位置的情况下，在链表上插入、删除数据是特别快的。但是在链表上查找时间复杂度是O(n)，所以链表这种数据结构本身并不适合存储有序的数据。有没有什么办法可以优化一下呢？

> 一种思路是，我们可以使用索引的思想来加快查找速度，那么如何来设计索引呢？

> 可以从每两个节点中选取一个节点，形成索引链表，索引链表的节点多出一个“down”指针，指向下一级（只有一层索引的情况下，指向原数据链表，否则指向下一级索引链表）。这样查找速度就快了一倍，类似的，我们可以在此基础上，继续增加更多的索引链表。

> 索引链表从“上”到“下”，分别为k、k-1、... 1 级索引。查找时，从最大的索引链表开始，依次同下一个节点值进行比较，如果下一个节点不存在或者值大于查找值，则向下一级重复这个查找过程，直到找到数据或者下一级不存在时停止

## 性能

> 如果每k个节点提取一个索引，则一共有logkn 层
> 每层最多遍历k个节点，所以时间复杂度是O(klogkn)，k是一个常数且远小于n，所以是O(logn)
> 第一级索引节点数量是 n/k，第二级是 n/k2，以此类推，索引总数量是 n/k, n/k2, .... , k, 1，是一个等比数列，等于 (n-1)/(k-1)，所以空间复杂度是O(n)

In [1]:
import random


class SkipList:
    MAX_LEVEL = 16

    class Node:
        def __init__(self, data=None):
            self.data = data
            self.forwards = [None] * SkipList.MAX_LEVEL
    
    def __init__(self):
        self.head = SkipList.Node()
        self.level = 1
    
    def _random_level(self):
        level = 1
        for _ in range(SkipList.MAX_LEVEL - 1):
            if random.random() > 0.5:
                level += 1
        
        return min(self.level + 1, level)

    def search(self, data):
        cur = self.head
        for i in range(self.level, 0, -1):
            if cur.forwards[i-1] and cur.forwards[i-1].data < data:
                cur = cur.forwards[i-1]
        
        return cur.forwards[0] is not None and cur.forwards[0].data == data
    
    def add(self, data):
        level = self._random_level()
        pre = [None] * level
        cur = self.head
        for i in range(level, 0, -1):
            while cur.forwards[i-1] and cur.forwards[i-1].data < data:
                cur = cur.forwards[i-1]
            pre[i-1] = cur
        
        node = SkipList.Node(data)
        for i in range(level):
            node.forwards[i] = pre[i].forwards[i]
            pre[i].forwards[i] = node
        
        self.level = max(self.level, level)
    

    def delete(self, data):
        pre = [None] * self.level
        cur = self.head
        for i in range(self.level, 0, -1):
            while cur.forwards[i-1] and cur.forwards[i-1].data < data:
                cur = cur.forwards[i-1]
            pre[i-1] = cur
        
        if not pre[0].forwards[0] or pre[0].forwards[0].data != data:
            return False
        
        for i in range(self.level):
            if pre[i].forwards[i] and pre[i].forwards[i].data == data:
                node = pre[i].forwards[i]
                pre[i].forwards[i] = node.forwards[i]
                node.forwards[i] = None
        
        while self.level > 1 and not self.head.forwards[self.level - 1]:
            self.level -= 1

        return True

